In [1]:
from oSLRAU_run import get_data
from spn.structure.leaves.parametric.Parametric import Gaussian, In_Latent
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.io.Graphics import plot_spn
import numpy as np
from spn.algorithms.Inference import log_likelihood
from sklearn.model_selection import train_test_split
from spn.algorithms.oSLRAU import oSLRAU, oSLRAUParams
from spn.algorithms.RSPN import RSPN
from spn.algorithms.TransformStructure import Prune ,Prune_oSLRAU

from hmmlearn import hmm
from spn.algorithms.Inference import log_likelihood

In [2]:
# Set the mean and standard deviation of the Gaussian distribution
mean0 = 0.0 
std0 = 1.0  

mean1 = 10.0 
std1 = 1.0  

# Generate random samples from the Gaussian distribution
num_samples=1000

gen_data = [eval(f"np.random.normal(mean{i%2}, std{i%2})") for i in range(num_samples)]


print(gen_data[:6])

[-1.312927744354675, 9.548574698788444, -0.6570525135110558, 8.467681347599486, 0.5747912883108425, 11.850302596035458]


In [3]:
n_states = 2  # Number of hidden states
model = hmm.GaussianHMM(n_components=n_states)

In [4]:
# Reshape the observations into a 2D array (required by hmmlearn)
X = np.array(gen_data).reshape(-1, 1)

# Fit the model to the data
model.fit(X)

GaussianHMM(n_components=2)

In [33]:
def train_rspn(train_data, test_data, rspn=None):

    if len(train_data.shape) == 3: train_data = np.squeeze(train_data)
    if len(test_data.shape) == 3: test_data = np.squeeze(test_data)

    if rspn is None:
    
        rspn = RSPN(num_variables=num_variables, num_latent_variables=num_latent_variables, num_latent_values=num_latent_values)
        first_mini_batch = train_data[0:mini_batch_size]
        n = first_mini_batch.shape[1]
        print(n)
        context = [Gaussian] * int(n)
        ds_context = Context(parametric_types=context).add_domains(first_mini_batch[:, :num_variables])
        spn, initial_template_spn, top_spn = rspn.build_initial_template(first_mini_batch, ds_context, len_sequence_varies)

    
    no_of_minibatches = int(train_data.shape[0] / mini_batch_size)

    print(f"no of minibatches: {no_of_minibatches}")

    
    for i in range(1, no_of_minibatches):
        mini_batch = train_data[i * mini_batch_size: (i+1) * mini_batch_size]
    
        update_template = False
        
        if i % update_after_no_min_batches == 0:
            print(i)
            update_template = True
    
        template_spn = rspn.learn_rspn(mini_batch, update_template, oSLRAU_params, unroll, full_update, update_leaves,
                                       len_sequence_varies)
           
            
    test_ll = np.mean(rspn.log_likelihood(test_data, unroll, len_sequence_varies))


    return test_ll, rspn, template_spn

In [34]:
def final_ll(data, num_epochs=1, do_plot_spn=True):
    final_test_ll = 0
    rspn=None
    train_data, val_data= train_test_split(data, test_size=0.1, random_state=42)
    for i in range(num_epochs):
        final_test_ll, rspn, template_spn = train_rspn(train_data, val_data, rspn)        
        print(f"final_test_ll epoch {i+1}/{num_epochs}: {final_test_ll}")

    if do_plot_spn:
        plot_spn(template_spn, 'rspn_final_template.pdf')
    
    return rspn

In [39]:
num_time_steps = 5
num_seq = 10
rspn_data = np.array([model.sample(num_time_steps)[0] for _ in range(num_seq)])

In [40]:
rspn_data.shape

(10, 5, 1)

In [43]:
num_variables = num_time_steps*1
num_latent_variables = 2
num_latent_values = 2
unroll = 'backward'
full_update = False
update_leaves = True
len_sequence_varies = False
oSLRAU_params = oSLRAUParams(mergebatch_threshold=10, corrthresh=0.7, mvmaxscope=1, equalweight=True, currVals=True)


mini_batch_size = 3
update_after_no_min_batches = 2

In [44]:
rspn = final_ll(rspn_data)

5
==>> data.shape: (3, 5)
len(self.meta_types): 5
self.meta_types.values(): dict_values([<MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>])
len(self.meta_types.values()): 5
data[:, col]: [15.40884759  3.33400006  0.06995246]
==>> domain_values: [0.06995245615406409, 15.408847593773359]
feature_meta_type: MetaType.REAL
==>> domain_values: [0.06995245615406409, 15.408847593773359]
len(self.meta_types.values()): 5
data[:, col]: [ 4.99424095 -2.87386971 10.5811393 ]
==>> domain_values: [-2.8738697118586733, 10.581139301123716]
feature_meta_type: MetaType.REAL
==>> domain_values: [-2.8738697118586733, 10.581139301123716]
len(self.meta_types.values()): 5
data[:, col]: [10.41450541  2.52336768  3.08404566]
==>> domain_values: [2.523367677506619, 10.414505414480985]
feature_meta_type: MetaType.REAL
==>> domain_values: [2.523367677506619, 10.414505414480985]
len(self.meta_types.values()): 5
data[:, col]: [12.27821795  4.34572398  1.14920196]
==>

IndexError: tuple index out of range